# Setup

## Load Packages

In [451]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import openai
import re
import time
from scripts.prepare_data_helpers import prepare_examples, create_query
import spacy
import textacy
from dotenv import load_dotenv

load_dotenv()
nlp = spacy.load('en_core_web_sm')

## Prepare Data

In [2]:
train1 = pd.read_csv('../../e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../../e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
dev = pd.read_csv('../../e-SNLI/dataset/esnli_dev.csv')
test = pd.read_csv('../../e-SNLI/dataset/esnli_test.csv')

train = train.dropna(subset=['Sentence1', 'Sentence2', 'Explanation_1'])
train = train.reset_index(drop=True)

np.random.seed(12345) # seed for numpy package
test_indices = list(np.random.choice(test.index.values, size=1000, replace=False))
test = test.loc[test_indices]
test = test.reset_index(drop=True)

In [68]:
gpt_pred = pd.read_csv('../output_data/test_predictions.csv', sep=';')

In [69]:
gpt_pred

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
0,0,4813951931.jpg#3r1e,entailment,A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.",A young man in an orange hat walks his *bicyc...,A man in an orange hat walks his *bike* down ...,9,8,...,"9,1,2","8,1",The young man in an orange hart is the man wal...,A *young* *man* in *an* *orange* *hat* *walks...,A *man* in *an* *orange* *hat* *walks* *his* ...,"1,2,4,5,6,7,8,9,10,12","1,5,4,3,11,9,8,7,6",Statement: A man in a suit and dirty dress-sho...,The man is young and the hat is orange.,entailment
1,1,4678320536.jpg#1r1c,contradiction,A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,A woman standing with *smile* *on* *her* *face...,A woman stands holding her baby and *crying.*,"4,5,6,7",7,...,13,5,the woman is either crying or has a smile,A *woman* standing with *smile* on her face an...,A *woman* stands holding her baby and *crying.*,"4,1","1,7",Statement: a white dog runs across the water.\...,"The woman is smiling and holding a wood, not a...",contradiction
2,2,661749711.jpg#2r1e,entailment,A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,A *dog* *jumps* to *catch* a red *ball* *outs...,the *dog* is *jumping.*,"2,7,8,1,4","3,1",...,"1,2","1,3","If the dog jumps to catch, the dog is jumping.",A dog *jumps* to *catch* a red ball outside.,the dog is *jumping.*,"2,4",3,Statement: A young child rides a bike with a b...,The statement is repeating the same information.,tautology
3,3,44904567.jpg#1r1c,contradiction,A man wielding an electric razor is gleefully ...,There is only one person present.,There cannot be one person present if one man ...,A *man* wielding an electric razor is gleefull...,There is only *one* *person* *present.*,"1,10,11","3,4,5",...,10,"3,2",It cannot be claimed that there is only one pe...,A *man* wielding an electric razor is gleefull...,There is *only* *one* *person* present.,"1,10,11","2,3,4",Statement: A young boy wearing safety glasses ...,There are two people present in the statement.,contradiction
4,4,32542645.jpg#0r1n,neutral,A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,A group of people stares at a wall that is fil...,A group of people *analyze* art together,{},4,...,{},"4,5",The group may have just been admiring as oppos...,A group of people stares at a wall that is fil...,A group of people *analyze* art *together*,{},"4,6",Statement: Young mother and child heading some...,Staring at a wall filled with drawings is a fo...,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2429284131.jpg#2r1c,contradiction,Three men are standing on stage performing.,Women are playing trumpets on the street.,"either there are men on stage, or women playin...",Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,...,1,0,There can be either men or women.,Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,Statement: Several basketball players are grab...,"The men are standing on stage performing, whil...",contradiction
996,996,4460943467.jpg#0r1e,entailment,3 young man in hoods standing in the middle of...,Three hood wearing people stand in a street.,3 Man in hoods are hood wearing people.,*3* young *man* in *hoods* standing in the mi...,Three *hood* *wearing* *people* stand in a st...,"2,0,4","1,2,3",...,"1,0,2,4,12","0,1,4,7",I

# Prepare Input Sentences

In [458]:
test = nlp('The girls are playing on the street')
test[1].head

playing

In [621]:
for test in textacy.extract.matches.token_matches(nlp("The dog is eating food and the child plays outside."), [{"DEP": "dobj"}]):
    print(test)

food


In [701]:

def extract_SVO(txt):
    res = list()
    doc = nlp(txt)

    for cur_nsubj in textacy.extract.matches.token_matches(doc, [{"DEP": "nsubj"}]):
        verbs = list()
        if cur_nsubj[0].head.pos_ == "AUX":
            for cur_verb in textacy.extract.matches.token_matches(doc, [{"POS": "VERB"}]):
                if cur_verb[0].head == cur_nsubj[0]:
                    verbs += [cur_verb[0]]
        else:
            verbs += [cur_nsubj[0].head]
        for cur_verb in verbs:
            dobjs = list(textacy.extract.matches.token_matches(doc, [{"DEP": "dobj"}]))
            if len(dobjs) == 0:
                res += [(cur_nsubj[0], cur_verb, None)]
            else:
                for cur_dobj in dobjs:
                    if cur_dobj[0].head == cur_verb:
                        res += [(cur_nsubj[0], cur_verb, cur_dobj[0])]
                    else:
                        res += [(cur_nsubj[0], cur_verb, None)]


    if len(res) == 0:
        cur_root = list(textacy.extract.matches.token_matches(doc, [{"DEP": "ROOT", "POS": "NOUN"}]))
        if len(cur_root) != 0:
            cur_root = cur_root[0][0]
            for cur_verb in textacy.extract.matches.token_matches(doc, [{"DEP": "acl"}]):
                if cur_verb[0].head == cur_root:
                    dobjs = list(textacy.extract.matches.token_matches(doc, [{"DEP": "dobj"}]))
                    if len(dobjs) == 0:
                        res += [(cur_root, cur_verb[0], None)]
                    for cur_dobj in dobjs:
                        if cur_dobj[0].head == cur_verb[0]:
                            res += [(cur_root, cur_verb[0], cur_dobj[0])]
                        else:
                            res += [(cur_root, cur_verb[0], None)]

    return res

for i in range(20):
    test = gpt_pred.Sentence1.iloc[i]
    #print(test)
    print(len(extract_SVO(test)))

1
2
1
2
1
0
1
1
1
2
1
1
1
1
1
1
1
1
1
1


In [706]:
gpt_pred['S1_SVO'] = [extract_SVO(gpt_pred.Sentence1.iloc[i]) for i in range(gpt_pred.shape[0])]
gpt_pred['S2_SVO'] = [extract_SVO(gpt_pred.Sentence2.iloc[i]) for i in range(gpt_pred.shape[0])]

In [708]:
gpt_pred[["S1_SVO", "S2_SVO", "Sentence1", "Sentence2"]]

,S1_SVO,S2_SVO,Sentence1,Sentence2
0,"[(man, walks, bicycle)]","[(man, walks, bike)]",A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...
1,"[(woman, holding, wood), (she, holding, wood)]","[(woman, stands, None)]",A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.
2,"[(dog, jumps, None)]","[(dog, jumping, None)]",A dog jumps to catch a red ball outside.,the dog is jumping.
3,"[(man, wielding, razor), (man, wielding, None)]",[],A man wielding an electric razor is gleefully ...,There is only one person present.
4,"[(group, stares, None)]","[(group, analyze, art)]",A group of people stares at a wall that is fil...,A group of people analyze art together
...,...,...,...,...
995,"[(men, standing, None)]","[(Women, playing, trumpets)]",Three men are standing on stage performing.,Women are playing trumpets on the street.
996,"[(man, standing, None)]","[(people, stand, None)]",3 young man in hoods standing in the middle of...,Three hood wearing people stand in a street.
997,"[(woman, torn, stockings)]","[(man, going, None)]","A woman in skirt, torn stockings and combat bo...",The man and woman are going to a movie in the ...
998,"[(men, shining, shoes)]",[],Two men shining peoples shoes.,Two men shining shoes.


In [635]:
nlp("A man in a plaid shirt is walking on a street where there are advertisements in Chinese.")[1].head

walking

In [587]:
list(textacy.extract.subject_verb_object_triples(nlp("A woman stands with a smile on her face and a girl is holding a wood in her hand.")))[0]

SVOTriple(subject=[woman], verb=[stands], object=[wood])

In [569]:

def extract_SVO(text):
    tuples = textacy.extract.subject_verb_object_triples(text)
    tuples_to_list = list(tuples)
    return tuples_to_list

for i in range(20):
    print(i)
    test = nlp(gpt_pred.Sentence1.iloc[i])
    print(test)
    print(extract_SVO(test))

0
A young man in an orange hat walks his bicycle down a street.
[SVOTriple(subject=[man], verb=[walks], object=[bicycle])]
1
A woman standing with smile on her face and she is holding a wood in her hand
[SVOTriple(subject=[woman, she], verb=[is, holding], object=[wood])]
2
A dog jumps to catch a red ball outside.
[SVOTriple(subject=[dog], verb=[jumps], object=[to, catch, a, red, ball, outside])]
3
A man wielding an electric razor is gleefully shaving away another man's hair.
[]
4
A group of people stares at a wall that is filled with drawings in a building.
[]
5
A snowboarder with a maroon coat sails through the air.
[]
6
A man in a blue jacket leans on a post and talks on the phone while an old man carries yellow bags.
[SVOTriple(subject=[man], verb=[carries], object=[bags])]
7
A man in a plaid shirt is walking on a street where there are advertisements in Chinese.
[]
8
Four women carrying shoulder bags are walking down a city sidewalk.
[]
9
A young man in a white hat is playing golf 

# Analyse Predictions

In [4]:
docs = [nlp(test.iloc[i].Explanation_1) for i in range(20)]

In [5]:
for doc in docs:
    for chunk in doc.noun_chunks:
        print(chunk.text, '//', chunk.root.text, '//', chunk.root.dep_, '//', chunk.root.head.text)
    print('-----------------------------------------')

A bicycle // bicycle // nsubj // are
a bike // bike // conj // bicycle
the same object // object // attr // are
-----------------------------------------
a woman // woman // nsubj // has
a smile // smile // dobj // has
her face // face // pobj // on
she // she // nsubj // crying
-----------------------------------------
Jumping // Jumping // nsubj // is
an action // action // attr // is
a ball // ball // dobj // catch
-----------------------------------------
one person // person // attr // be
one man // man // nsubj // shaving
another man's hair // hair // dobj // shaving
-----------------------------------------
they // they // nsubj // analyze
the art // art // dobj // analyze
-----------------------------------------
A snowboarder // snowboarder // nsubj // is
a person // person // attr // is
a snowboarder // snowboarder // nsubj // sails
the air // air // pobj // through
they // they // nsubj // snowboarding
a person // person // nsubj // snowboarding
-----------------------------

In [6]:
for i in range(10):
    doc = nlp(test.iloc[i].Explanation_1)
    html = spacy.displacy.render(doc, style='dep')
    with open('data_vis' + str(i) + '.html', 'w') as f:
        f.write(html)

TypeError: write() argument must be str, not None

In [696]:
doc = nlp(gpt_pred.iloc[44].Sentence1)
html = spacy.displacy.render(doc, style='dep', jupyter=False)
with open('data_vis' + 'test' + '.html', 'w') as f:
    f.write(html)

In [514]:
spacy.explain("advcl")

'adverbial clause modifier'

In [37]:
train.Sentence2.iloc[3]

'A person is outdoors, on a horse.'

In [38]:
train.Explanation_1.iloc[3]

'Just because they are smiling and waving at a camera does not imply their parents or anyone is anyone behind it'

In [106]:
gpt_pred.head()

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
0,0,4813951931.jpg#3r1e,entailment,A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.",A young man in an orange hat walks his *bicyc...,A man in an orange hat walks his *bike* down ...,9,8,...,"9,1,2","8,1",The young man in an orange hart is the man wal...,A *young* *man* in *an* *orange* *hat* *walks...,A *man* in *an* *orange* *hat* *walks* *his* ...,"1,2,4,5,6,7,8,9,10,12","1,5,4,3,11,9,8,7,6",Statement: A man in a suit and dirty dress-sho...,The man is young and the hat is orange.,entailment
1,1,4678320536.jpg#1r1c,contradiction,A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,A woman standing with *smile* *on* *her* *face...,A woman stands holding her baby and *crying.*,"4,5,6,7",7,...,13,5,the woman is either crying or has a smile,A *woman* standing with *smile* on her face an...,A *woman* stands holding her baby and *crying.*,"4,1","1,7",Statement: a white dog runs across the water.\...,"The woman is smiling and holding a wood, not a...",contradiction
2,2,661749711.jpg#2r1e,entailment,A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,A *dog* *jumps* to *catch* a red *ball* *outs...,the *dog* is *jumping.*,"2,7,8,1,4","3,1",...,"1,2","1,3","If the dog jumps to catch, the dog is jumping.",A dog *jumps* to *catch* a red ball outside.,the dog is *jumping.*,"2,4",3,Statement: A young child rides a bike with a b...,The statement is repeating the same information.,tautology
3,3,44904567.jpg#1r1c,contradiction,A man wielding an electric razor is gleefully ...,There is only one person present.,There cannot be one person present if one man ...,A *man* wielding an electric razor is gleefull...,There is only *one* *person* *present.*,"1,10,11","3,4,5",...,10,"3,2",It cannot be claimed that there is only one pe...,A *man* wielding an electric razor is gleefull...,There is *only* *one* *person* present.,"1,10,11","2,3,4",Statement: A young boy wearing safety glasses ...,There are two people present in the statement.,contradiction
4,4,32542645.jpg#0r1n,neutral,A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,A group of people stares at a wall that is fil...,A group of people *analyze* art together,{},4,...,{},"4,5",The group may have just been admiring as oppos...,A group of people stares at a wall that is fil...,A group of people *analyze* art *together*,{},"4,6",Statement: Young mother and child heading some...,Staring at a wall filled with drawings is a fo...,entailment


In [180]:
test = dict()
test['hi'] = ['vhi']
test['ljsk'] = [23]
test['hi'] += [12]
test

{'hi': ['vhi', 12], 'ljsk': [23]}

In [183]:
'sldfkj' in test.keys()

False

In [230]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
doc[2].dep_

'ROOT'

In [301]:
doc = nlp("Apple does want to buy U.K. startup for $1 billion")

neg_root = None
for token in doc:
    if token.dep_ == 'ROOT':
        cur_root = token.lemma_
    if token.dep_ == 'neg' and token.head.dep_ == 'ROOT':
        neg_root = token.lemma_ + " " + token.head.lemma_
        print(neg_root)
    if neg_root:
        cur_root = neg_root
print(cur_root)

want


In [444]:
n = gpt_pred.shape[0]
expls = [nlp(gpt_pred.pred_explanation.iloc[i]) for i in range(n)]
sent1s = [nlp(gpt_pred.Sentence1.iloc[i]) for i in range(n)]
sent2s = [nlp(gpt_pred.Sentence2.iloc[i]) for i in range(n)]
input_sentence_root = [[sent1s[j][i].lemma_ for i in range(len(sent1s[j])) if sent1s[j][i].dep_=="ROOT"] + [sent2s[j][i].lemma_ for i in range(len(sent2s[j])) if sent2s[j][i].dep_=="ROOT"] for j in range(n)]
input_sentence_subj = [[sent1s[j][i].lemma_ for i in range(len(sent1s[j])) if sent1s[j][i].dep_=="ROOT"] + [sent2s[j][i].lemma_ for i in range(len(sent2s[j])) if sent2s[j][i].dep_=="ROOT"] for j in range(n)]

In [446]:
input_sentence_root[0]

['walk', 'walk']

In [447]:
verbs_sentence = list()

cur_index = 0
verb_dict = dict()
root_term_dict = dict()

# loop through each row in the test data
for expl, input_sent in zip(expls, input_sentence_root):
    cur_verbs = list()
    neg_root = None
    contains_different = False
    root_in_input_sentence_lemmata = False

    # loop through each token in current explanation
    for token in expl:
        if token.lemma_ == 'different':
            contains_different = True
        if token.dep_ == 'ROOT':
            cur_root = token.lemma_
            if cur_root in input_sent:
                root_in_input_sentence_lemmata = True
        if token.dep_ == 'neg' and token.head.dep_ == 'ROOT':
            neg_root = token.lemma_ + " " + token.head.lemma_
        if token.pos_ == 'VERB' and token.lemma_ not in input_sent:
            cur_verbs += [token.lemma_,]
            if token.lemma_ not in verb_dict.keys():
                verb_dict[token.lemma_] = [cur_index]
            else:
                verb_dict[token.lemma_] += [cur_index]
        if token.dep_ == 'ROOT':
            cur_root = token.lemma_
    if neg_root:
        cur_root = neg_root
    if contains_different:
        cur_root += " different"
    if root_in_input_sentence_lemmata:
        cur_root = "root is in input sentence"
    if cur_root not in root_term_dict.keys():
        root_term_dict[cur_root] = [cur_index]
    else:
        root_term_dict[cur_root] += [cur_index]
    verbs_sentence += [cur_verbs, ]
    cur_index += 1
verbs_sentence = pd.Series(verbs_sentence)
verbs = pd.Series([i for b in map(lambda x:[x] if not isinstance(x, list) else x, verbs_sentence.values) for i in b])

In [448]:
pd.Series(root_term_dict)

be                           [0, 4, 11, 27, 45, 47, 53, 65, 83, 84, 86, 88,...
smile                                                                 [1, 188]
repeat                                                                     [2]
root is in input sentence    [3, 14, 16, 25, 39, 40, 42, 51, 56, 61, 62, 63...
refer                                             [5, 472, 476, 667, 869, 966]
                                                   ...                        
sell                                                                     [883]
not assume                                                               [894]
not mean different                                                       [972]
restate                                                                  [973]
cause                                                                    [994]
Length: 97, dtype: object

In [449]:
gpt_pred[["Sentence1", "Sentence2", "pred_explanation"]].iloc[root_term_dict['root is in input sentence']]

,Sentence1,Sentence2,pred_explanation
3,A man wielding an electric razor is gleefully ...,There is only one person present.,There are two people present in the statement.
14,A woman in a striped shirt is walking a horse ...,A woman in a striped shirt is running with a d...,The woman cannot be walking a horse and runnin...
16,The basketball player in red is challenging th...,A basketball player in red is trying to grab t...,The basketball player in red is challenging th...
25,Two people are playing wiffle ball in a suburb...,Two men are playing basketball.,"The two people are playing wiffle ball, which ..."
39,a man giving two thumbs up to a crowd.,The crowd watches him,For the man to give two thumbs up to the crowd...
...,...,...,...
974,A man wearing a snorkel and goggles gives a th...,Two guys are on a lake.,A lake is a body of water and snorkeling requi...
975,Two mountaineers are walking across a snowcapp...,Two mountaineers sitting at the beach.,Mountaineers cannot be walking across a snowca...
985,Two small children walk away in a field.,"Two small children are sitting down, eating ic...",The two children are walking away in one state...
992,A woman examines a poster mounted on the wall ...,There is a woman under fluorescent lights.,If a woman is examining a poster mounted on th...


In [419]:
gpt_pred.iloc[root_term_dict['wear']]

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
26,26,4824252063.jpg#1r1e,entailment,A man with a latex glove and black t-shit is s...,a man standing in front of a van,A man standing in front of a Chevy van is in f...,A man with a latex glove and black t-shit is ...,a man *standing* in front of a *van*,"16,15,10","7,2",...,"1,2,3,4,5,15,16","1,7",A Chevy van is a specific type of a van.,A man with a latex glove and black t-shit is ...,a man standing in front of a *van*,"15,16",7,Statement: A soccer team dressed in blue has c...,The man is wearing a latex glove and black t-s...,entailment
79,79,2324374253.jpg#4r1c,contradiction,Two kids in their pajamas.,Two children are wearing bright bathing suits ...,bathing suits are not pajamas,Two kids in their *pajamas.*,Two children are wearing bright *bathing* *sui...,4,"6,5",...,4,"5,6",children can't be wearing bathing suites if th...,Two *kids* *in* their *pajamas.*,Two *children* are wearing bright *bathing* *s...,"4,2,1","5,6,1",Statement: A man sitting in between two roads ...,"The two kids are wearing pajamas, which are no...",contradiction
133,133,4429727181.jpg#0r1e,entailment,A young male wearing a blue hat turned backwar...,A man is wearing a hat.,"Young male is a man, and blue hat is a type of...",A *young* *male* wearing a *blue* *hat* turne...,A *man* is wearing a *hat.*,"1,2,5,6","1,5",...,2,1,A young man wearing a blue hat turned backward...,A young *male* *wearing* a blue *hat* turned ...,A *man* is *wearing* a *hat.*,"3,6,2","3,5,1","Statement: A woman in a black shirt and jeans,...","If there is a young male wearing a blue hat, t...",entailment
154,154,5745422329.jpg#2r1c,contradiction,"One man wearing a pink and blue striped shirt,...",Nobody has a shirt on,The man can't have a shirt while not wearing a...,One man *wearing* a pink and blue striped *sh...,*Nobody* has a shirt on,"2,8",0,...,"0,1,2,3,4,5,6,7,8","0,1,2,3,4","Nobody refers to not a single person, and one ...",*One* *man* wearing a pink and blue striped s...,*Nobody* has a shirt on,"1,0",0,Statement: A boy jumping off a yellow spring b...,The man is wearing a pink and blue striped shirt.,contradiction
204,204,4081678254.jpg#0r1c,contradiction,"A man wearing dark jeans, an orange shirt, and...",The woman is wearing a red top.,A woman wearing a red top cannot transform int...,"A *man* wearing dark jeans, an *orange* *shir...",The *woman* is wearing a *red* *top.*,"1,6,7","1,5,6",...,"6,7,1","5,6,1",The woman can't have on a red top and blue top.,"A man wearing dark jeans, an orange shirt, an...",The *woman* is wearing a *red* *top.*,"23,27,26","1,5,6",Statement: Men looking under the hood of a dra...,"The woman is wearing a blue top, not a red top.",contradiction
282,282,3773298212.jpg#0r1c,contradiction,A man dressed in a brown shirt and tan shorts ...,The man is wearing tan pants.,Shorts and pants are not the same thing.,A man dressed in a brown shirt and tan *shorts...,The man is wearing tan *pants.*,9,5,...,9,5,Shorts can not be pants.,A man dressed in a brown shirt and tan *shorts...,The man is wearing tan *pants.*,9,5,Statement: Two girls going down a water slide....,"The man is wearing shorts, not pants.",contradiction
335,335,4705913066.jpg#4r1c,contradiction,A man in a suit speaking to a seated woman.,A man in a costume speaking to another man.,"If he is in a suit talking to a woman, he is n...",A man in a *suit* speaking to a seated *woman.*,A man in a *costume* speaking to another *man.*,"4,9","4,8",...,"9,5,1","5,8,1",A MAN AND A WOMAN CANNOT BE SAME,A man in a suit speaking to a seated *woman.*,A man in a costume speaking to another *man.*,9,8,Statement: Surfer in wetsuit riding a wave.\nS...,The man is wearin

In [344]:
gpt_pred.pred_explanation.iloc[root_term_dict['be']]

0                The man is young and the hat is orange.
3         There are two people present in the statement.
4      Staring at a wall filled with drawings is a fo...
11     Cooking on a grill is a rewording of having a ...
27     Watching a golf putt on the set of Sports Cent...
                             ...                        
982    Hanging on to the bottom of an airplane and dr...
983    "Playing catch" is an activity that children c...
986    Tom and Sue are two people, and they are playi...
992    If a woman is examining a poster mounted on th...
997    The statement does not give any indication tha...
Name: pred_explanation, Length: 241, dtype: object

In [356]:
root_words = list()
n = gpt_pred.shape[0]
cur_index = 0
root_term_dict = dict()
expls = [nlp(gpt_pred.pred_explanation.iloc[i]) for i in range(n)]
for expl in expls:
    for token in expl:
        if token.lemma_ == 'different':
            root_words += [token.head.text]
            if token.head.lemma_ not in root_term_dict.keys():
                root_term_dict[token.head.lemma_] = [cur_index]
            else:
                root_term_dict[token.head.lemma_] += [cur_index]
    cur_index += 1
verbs_sentence = pd.Series(root_words)

In [373]:
for k, v in zip(root_term_dict.keys(), root_term_dict.values()):
    print('=========================')
    print(k)
    try:
        print(gpt_pred.gold_label.iloc[v].value_counts()['contradiction'] / gpt_pred.gold_label.iloc[v].value_counts().sum())
    except:
        print(0)

action
0.9166666666666666
age
0
be
0.7692307692307693
activity
0.88
scene
1.0
setting
0
thing
1.0
scenario
1.0
game
1.0
location
1.0
sport
1.0
term
0
people
1.0
description
1.0
shot
1.0
word
0
item
1.0
background
1.0
time
1.0
environment
0.3333333333333333
same
0
situation
1.0
place
1.0
type
1.0
equipment
1.0
clothing
1.0
animal
1.0
country
0
emotion
1.0
instrument
1.0
dog
1.0
mode
1.0
technique
1.0


In [375]:
gpt_pred.iloc[root_term_dict['action']]

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
8,8,4547257158.jpg#2r1c,contradiction,Four women carrying shoulder bags are walking ...,Four women are running away from a man in the ...,Running and walking are two different gaits.,Four women carrying shoulder bags are *walking...,Four women are *running* away from a man in th...,6,3,...,6,3,walking not running,Four women carrying shoulder bags are *walking...,Four women are *running* away from a man in th...,6,3,"Statement: people playing, basketball ball in ...",The statements are describing two different ac...,contradiction
58,58,4604977910.jpg#1r1c,contradiction,A woman standing in front of a large tank at a...,A woman is tying her shoes,The woman can't be tying her shoes while stand...,A woman *standing* in front of a large tank at...,A woman is *tying* her shoes,2,3,...,"12,13","3,4,5",She cannot be tying her shoes while she turns ...,A woman standing in front of a large tank at a...,A woman is *tying* *her* *shoes*,"12,13","3,4,5",Statement: Man wearing black shirt throwing an...,The statement does not describe the woman stan...,contradiction
153,153,3519155763.jpg#3r1c,contradiction,A dog opens its mouth to catch a blue and yell...,The pet is chewing on a bone,Bones are not blue and yellow.,A dog opens its mouth to catch a *blue* *and* ...,The pet is chewing on a *bone*,"10,11,9,8",6,...,"11,8,10",6,A pet is not necessarily a dog. The pet is ei...,A *dog* *opens* *its* *mouth* *to* *catch* a b...,The *pet* *is* *chewing* on a bone,"1,2,3,4,5,6","1,2,3",Statement: One dog is chasing another dog that...,opening mouth to catch a toy and chewing on a ...,contradiction
216,216,3025315215.jpg#0r1c,contradiction,a brown dog coming out of an orange and yellow...,A brown dog dancing out of a orange and yellow...,The dog cannot be both coming and dancing out ...,a brown *dog* *coming* out of an orange and ye...,A brown *dog* *dancing* out of a orange and ye...,"3,2","3,2",...,3,3,A dog dancing out could not have been coming out.,a brown dog *coming* *out* of an orange and ye...,A brown dog *dancing* *out* of a orange and ye...,"3,4","3,4",Statement: Girl with crossed arms stands in st...,The sentences have different actions (coming o...,contradiction
219,219,3246016210.jpg#4r1c,contradiction,One man cutting through ice on a cold day.,The man is running around his house.,one can not be cutting through ice and be runn...,One man *cutting* *through* *ice* on a cold day.,The man is *running* around his *house.*,"2,3,4","3,6",...,"2,3,4","3,4,5,6",Cutting through ice isnt running around the ho...,One man *cutting* *through* *ice* on a cold day.,The man is *running* *around* his *house.*,"2,3,4","3,4,6",Statement: Two women in their karate uniforms ...,Cutting through ice and running around a house...,contradiction
364,364,7249180494.jpg#0r1c,contradiction,A professional swimmer spits water out after s...,The swimmer is eating a toasted marshmallow.,The swimmer is either swimming or eating a mar...,A professional swimmer spits water out after *...,The swimmer is *eating* a toasted marshmallow.,7,3,...,3,3,"If one spits water, they can not be eating a t...",A professional swimmer *spits* *water* out aft...,The swimmer is *eating* *a* *toasted* *marshma...,"3,4","4,3,5,6",Statement: These two ladies are working in the...,Spitting water out and eating a toasted marshm...,contradiction
385,385,3836884647.jpg#3r1c,contradiction,A man walks by a building at night.,A man is sitting in the sun.,The man cannot simultaneously be sitting and walk,A man *walks* by a building at night.,A man is *sitting* in the sun.,2,3,...,"2,7","3,6",A MAN EITHER WALKS OR HE WILL BE SITTING IN TH...,A man *walks* by a building at

In [405]:
gpt_pred.iloc[608].pred_explanation

'The woman is wearing dark brown leather jacket, sunglasses, brown leather boots, and a silver designer handbag, which implies that she is nicely dressed.'